# Neural architecture search for Multi-Layer Perceptron

### First defining the parameters to create a dictionary file containing a primary key for all possible combination of parameters


In [1]:
from parameters import *

In [2]:

def vocab_dict():
    vocab = {}
    
    for i in range(len(nodes)):
        for j in range(len(activations)):
            
            layer_id = len(activations) * i + j + 1
            
            vocab[layer_id] = (nodes[i], activations[j])
    
    vocab[len(vocab) + 1] = (('dropout'))
    
    if target_classes == 2:
        vocab[len(vocab) + 1] = (target_classes - 1, 'sigmoid')
    else:
        vocab[len(vocab) + 1] = (target_classes, 'softmax')
    return vocab

vocab = vocab_dict()
print('vocab size : ' ,len(vocab))

final_vocab_length = len(vocab)

# The final id of the vocabulary will always be the size of output classes and a softmax layer

print('Final layer id : ',vocab[final_vocab_length])

# the semi-final layer 
print("Second last layer of the vocabulary : ",vocab[final_vocab_length-1])

vocab size :  23
Final layer id :  (3, 'softmax')
Second last layer of the vocabulary :  dropout


In [3]:
## Step 2: sample architecture from the vocabulary according to the size of the defined model

final_layer_id = len(vocab)
dropout_id = final_layer_id -1
print('final_layer_id',final_layer_id)
print('dropout_id:',dropout_id)

vocab_idx = [0]+list(vocab.keys())
print('vocab ids:',vocab_idx)

final_layer_id 23
dropout_id: 22
vocab ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [4]:
from keras.preprocessing.sequence import pad_sequences

samples = []
seed = []

sequence = pad_sequences([seed],maxlen = 10-1,padding = 'post')
print(sequence)

2023-11-12 12:23:58.658630: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-12 12:23:58.726571: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-12 12:23:58.727803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 12:24:00.204703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[[0 0 0 0 0 0 0 0 0]]


In [5]:
sequence = sequence.reshape(1,1,9)
print(sequence)

[[[0 0 0 0 0 0 0 0 0]]]


In [6]:
# ## Define a LSTM model to choose between sequences?
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# 
# model = keras.Sequential()
# model.add(layers.LSTM(controller_input_shape, input_shape=(None, 28)))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(10))
# print(model.summary())

In [7]:
controller_classes = len(vocab) + 1
print(controller_classes)


24


In [8]:

from keras.layers import Input,LSTM,Dense
from keras.models import Model

main_input = Input(
    shape = controller_input_shape,
    batch_size = controller_batch_size,
    name = 'main_input')

x = LSTM(controller_lstm_dim,
         return_sequences = True)(main_input)

main_output = Dense(controller_classes,
                    activation = 'softmax',
                    name = 'main_output')(x)

model = Model(inputs = [main_input],
              outputs = [main_output])

print(model.summary())


2023-11-12 12:24:02.032602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-12 12:24:02.033800: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 main_input (InputLayer)     [(32, 1, 9)]              0         
                                                                 
 lstm (LSTM)                 (32, 1, 100)              44000     
                                                                 
 main_output (Dense)         (32, 1, 24)               2424      
                                                                 
Total params: 46424 (181.34 KB)
Trainable params: 46424 (181.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [2]:

from keras.layers import Input,LSTM,Dense
from keras.models import Model

main_input = Input(
    shape = controller_input_shape,
    batch_size = controller_batch_size,
    name = 'main_input')

x = LSTM(controller_lstm_dim,
         return_sequences = True)(main_input)

main_output = Dense(controller_classes,
                    activation = 'softmax',
                    name = 'main_output')(x)

model = Model(inputs = [main_input],
              outputs = [main_output])

print(model.summary())



2023-11-13 11:20:17.717501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 11:20:18.104269: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 11:20:18.106781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 11:20:19.978903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


NameError: name 'controller_input_shape' is not defined

In [9]:
print(number_of_samples)
print(max_len)

10
10


In [10]:
import numpy as np
seq_data = []

while len(samples) < number_of_samples :
    seed = []
    while len(seed) < max_len:
        sequence = pad_sequences([seed],maxlen = 10-1,padding = 'post')
        sequence = sequence.reshape(1,1,max_len-1)
        # print(sequence.shape)
        
        probab = model.predict(sequence)
        probab = probab[0][0]
        next = np.random.choice(vocab_idx,size=1, p = probab)[0]

        if next == dropout_id and len(seed) == 0:
            continue
        if next == final_layer_id and len(seed) == 0:
            continue
        if next == final_layer_id:
            seed.append(next)
            break
        if len(seed) == max_len - 1:
            seed.append(final_layer_id)
            break
        if not next == 0:
            seed.append(next)
    if seed not in seq_data:
        samples.append(seed)
        seq_data.append(seed)
        # print(probab)

1/1 [==============================] - 0s 25ms/step


In [11]:
print(samples)

[[8, 22, 3, 17, 21, 20, 8, 19, 15, 23], [3, 6, 15, 4, 6, 7, 12, 9, 9, 23], [10, 15, 17, 11, 8, 23], [2, 22, 11, 14, 15, 8, 1, 14, 7, 23], [1, 8, 15, 9, 23], [8, 3, 9, 6, 15, 21, 4, 1, 1, 23], [13, 3, 18, 23], [17, 22, 11, 15, 12, 14, 21, 14, 17, 23], [20, 21, 12, 1, 17, 20, 10, 6, 8, 23], [5, 14, 6, 17, 7, 3, 20, 14, 15, 23]]


In [12]:
# probab = probab[0][0]
# print(probab)

In [13]:
# import numpy as np
# next = np.random.choice(vocab_idx,size=1, p = probab)[0]
# print(next)

In [14]:
# seed.append(next)
# print(seed)

In [15]:
# sequence = pad_sequences([seed],maxlen=max_len-1,padding='post')
# print(sequence)

In [16]:
# sequence = sequence.reshape(1, 1, max_len - 1)
# print(sequence)

In [1]:
probab = model.predict(sequence)
print(probab)

NameError: name 'model' is not defined

In [ ]:
# next = np.random.choice(vocab_idx,size=1, p=probab)

In [19]:
pred_accuracies = []
for seq in samples:
    control_sequences = pad_sequences([seq],maxlen=max_len-1,padding='post')
    print("Control Sequences:", control_sequences)
    
    # xc = control_sequences[:, :-1]
    # print("Shape after slicing:", xc.shape)

    xc = control_sequences[:, :max_len-1].reshape(len(control_sequences), 1, max_len - 1)
    
    pred_accuracy = [x[0][0] for x in model.predict(xc)]
    pred_accuracies.append(pred_accuracy[0])
print(pred_accuracies)

Control Sequences: [[22  3 17 21 20  8 19 15 23]]
1/1 [==============================] - 0s 31ms/step
Control Sequences: [[ 6 15  4  6  7 12  9  9 23]]
1/1 [==============================] - 0s 26ms/step
Control Sequences: [[10 15 17 11  8 23  0  0  0]]
1/1 [==============================] - 0s 30ms/step
Control Sequences: [[22 11 14 15  8  1 14  7 23]]
1/1 [==============================] - 0s 43ms/step
Control Sequences: [[ 1  8 15  9 23  0  0  0  0]]
1/1 [==============================] - 0s 30ms/step
Control Sequences: [[ 3  9  6 15 21  4  1  1 23]]
1/1 [==============================] - 0s 34ms/step
Control Sequences: [[13  3 18 23  0  0  0  0  0]]
1/1 [==============================] - 0s 34ms/step
Control Sequences: [[22 11 15 12 14 21 14 17 23]]
1/1 [==============================] - 0s 29ms/step
Control Sequences: [[21 12  1 17 20 10  6  8 23]]
1/1 [==============================] - 0s 32ms/step
Control Sequences: [[14  6 17  7  3 20 14 15 23]]
1/1 [===========================

In [19]:
for i, sequence in enumerate(samples):
    